In [ ]:
import geopandas
import networkx as nx
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import copy

import os

import snman
from snman import osmnx_customized as oxc
from snman.constants import *

#PERIMETER = '_paper2_example'
PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)
outputs_path_accessibility = os.path.join(data_directory, 'outputs', '_accessibility_debug')

CRS_internal = 2056      # for Zurich

In [ ]:
population_sample = 0.05
statent_id = 68292494
distance_limit = 70*1000
destinations_sample = 0.05
cycling_speed_kmh = 18
walking_speed_kmh = 1.34 * 3.6
access_egress_detour_factor = 2
departure_time = datetime.datetime(2024, 2, 22, 18, 00)
min_travel_time = 5*60
min_euclidian_distance = 100

rebuild_inputs = True
STATE = 'before'
TIME = '18:00'

if STATE=='before':
    ln_desc = KEY_LANES_DESCRIPTION
else:
    ln_desc = KEY_LANES_DESCRIPTION_AFTER

In [ ]:
if rebuild_inputs and 1:
    r5_transit_network = r5py.TransportNetwork(
        os.path.join(outputs_path_accessibility, 'before_oneway_links_default.osm.pbf'),
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip'),
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'gtfs_fp2024_2024-06-27_mod.zip'),
    )

In [ ]:
# Load street graph
if rebuild_inputs and 1:
    G = snman.io.load_street_graph(
        os.path.join(outputs_path, 'G_edges.gpkg'),
        os.path.join(outputs_path, 'G_nodes.gpkg'),
        crs=CRS_internal
    )

In [ ]:
if rebuild_inputs:
    G_modes = {}
    L_modes = {}

In [ ]:
# create mode graphs for other modes
if rebuild_inputs:
    for mode in [MODE_CYCLING, MODE_FOOT]:
        print(f'Make street and lane graphs for {mode}')
        G_modes[mode] = copy.deepcopy(G)
        snman.street_graph.filter_lanes_by_modes(
            G_modes[mode], [mode], lane_description_key=ln_desc
        )
        L_modes[mode] = snman.lane_graph.create_lane_graph(
            G_modes[mode], lanes_attribute=ln_desc
        )

In [ ]:
if rebuild_inputs:
    # load the new car street graph created from MATSim output
    G_modes[MODE_PRIVATE_CARS] = snman.io.load_street_graph(
        os.path.join(outputs_path, f'tt_{STATE}_edges.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes.gpkg'),
        crs=CRS_internal,
        unstringify_attributes={'lanes': snman.space_allocation.space_allocation_from_string}
    )
    
    # we must avoid that the start/end point of trips are matched onto nodes that not accessible by car
    # therefore, we reduce the graph only to those links and nodes that are accessible by car
    snman.street_graph.filter_lanes_by_modes(
        G_modes[MODE_PRIVATE_CARS], [MODE_PRIVATE_CARS], lane_description_key='lanes'
    )

In [ ]:
if rebuild_inputs:
    L_modes[MODE_PRIVATE_CARS] = snman.lane_graph.create_lane_graph(
        G_modes[MODE_PRIVATE_CARS], lanes_attribute='lanes', cast_attributes={'matsim_tt_cars': f'car_{TIME}'}
    )

In [ ]:
if rebuild_inputs:
    # Import Statent grid
    statent = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
    )
    statent.rename(columns={'RELI': 'id'}, inplace=True)
    statent

In [ ]:
if rebuild_inputs:
    # read statpop that has been pre-joined with statent
    statpop_with_statent_ids = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
    )
    
    statpop_with_statent_ids.head()

In [ ]:
# filter statpop for residents within the given origin statent cell
statpop_filtered = statpop_with_statent_ids.query(f'statent_id == {statent_id}')
statpop_filtered = statpop_filtered.sample(frac=population_sample, random_state=0)
statpop_filtered

In [ ]:
# make a light version of statpop
statpop_reduced = statpop_filtered.reset_index()[[
    'record', 'age', 'sex', 'maritalstatus', 'residencepermit', 'residentpermit', 'statent_id', 'geometry'
]]
statpop_reduced.set_index('record', inplace=True)
statpop_reduced

In [ ]:
# filter statent for cells within a distance limit from the origin
origin = statent.set_index('id').geometry[statent_id]
origins = gpd.GeoDataFrame(
    {
        "id": [statent_id],
        "geometry": [origin]
    },
    crs="EPSG:2056",
)
statent_filtered = statent[statent['geometry'].within(
    origin.buffer(distance_limit)
)]
statent_filtered = gpd.GeoDataFrame(statent_filtered)
statent_filtered

In [ ]:
# make a sample of statpop cells but inlcude the origin cell in every case
statent_filtered = pd.concat([
    statent_filtered[statent_filtered['id'] == statent_id],
    statent_filtered[statent_filtered['id'] != statent_id].sample(frac=destinations_sample, random_state=0)
    ])

In [ ]:
# match statent points to lanegraph nodes
# TODO: do this only once

for mode in [MODE_PRIVATE_CARS, MODE_CYCLING, MODE_FOOT]:

    # match statent points to nearest nodes
    nodes = oxc.nearest_nodes(
        L_modes[mode],
        list(map(lambda geom: geom.x, statent_filtered.geometry)),
        list(map(lambda geom: geom.y, statent_filtered.geometry)),
        return_dist=True
    )

    statent_filtered[[f'closest_node_{mode}', f'egress_{mode}']] = list(zip(*nodes))

statent_filtered

In [ ]:
tt_matrices = {}

In [ ]:
# calculate transit travel time matrix using R5
tt_computer_transit = r5py.TravelTimeMatrixComputer(
    r5_transit_network,
    transport_modes=[r5py.TransportMode.TRANSIT],
    origins=origins,
    destinations=statent_filtered,
    departure=departure_time,
    snap_to_network=True
)
tt_matrices[MODE_TRANSIT] = tt_computer_transit.compute_travel_times()
tt_matrices[MODE_TRANSIT].rename(columns={
    'travel_time': 'travel_time_transit',
    'from_id': 'from_cell',
    'to_id': 'to_cell'
}, inplace=True)
# convert from minutes to seconds
tt_matrices[MODE_TRANSIT]['travel_time_transit'] *= 60
tt_matrices[MODE_TRANSIT]['travel_time_egress_transit'] = 0

tt_matrices[MODE_TRANSIT]

In [ ]:
# calculate shortest paths to all destinations in networkx
for mode in [MODE_PRIVATE_CARS, MODE_FOOT, MODE_CYCLING]:

    print(mode)

    origin_cell = statent_filtered[statent_filtered['id'] == statent_id]
    origin_node = min(origin_cell[f'closest_node_{mode}'])
    access_cost = min(origin_cell[f'egress_{mode}'])
    print(origin_node, access_cost)
    
    if mode == MODE_PRIVATE_CARS:
        weight = 'matsim_tt_cars'
    else:
        weight = f'cost_{mode}'
    
    # calculate cost to all other nodes
    dijkstra_path_costs = nx.single_source_dijkstra_path_length(L_modes[mode], origin_node, weight=weight)
    
    # convert the dijkstra path costs dict to a dataframe like from R5
    tt_matrices[mode] = pd.DataFrame(list(dijkstra_path_costs.items()), columns=['to_node', f'path_length_{mode}'])
    tt_matrices[mode]['from_node'] = origin_node
    
tt_matrices[MODE_PRIVATE_CARS]

In [ ]:
# merge the tt matrix of transit on cell_ids
statent_filtered_2 = pd.merge(
    statent_filtered, tt_matrices[MODE_TRANSIT],
    left_on='id', right_on='to_cell', how='left'
)
statent_filtered_2.drop(columns=['from_cell', 'to_cell'], inplace=True)
statent_filtered_2[f'travel_time_total_{MODE_TRANSIT}'] = statent_filtered_2[f'travel_time_{MODE_TRANSIT}']
statent_filtered_2

In [ ]:
# merge the other tt matrices on node ids
for mode in [MODE_PRIVATE_CARS, MODE_FOOT, MODE_CYCLING]:

    # merge the travel time matrices with the statent dataset
    # such that every destination has the cost of reaching it
    statent_filtered_2 = pd.merge(
        statent_filtered_2, tt_matrices[mode],
        left_on=f'closest_node_{mode}', right_on='to_node', how='left'
    )
    statent_filtered_2.drop(columns=['from_node', 'to_node'], inplace=True)
    
    origin_cell = statent_filtered[statent_filtered['id'] == statent_id]
    access_cost = min(origin_cell[f'egress_{mode}'])
    
    if mode == MODE_FOOT:
        speed_factor = walking_speed_kmh / 3.6
    elif mode == MODE_CYCLING:
        speed_factor = cycling_speed_kmh / 3.6
    else:
        speed_factor = 1
    
    # calculate total travel time by adding access and egress cost and considering speed factors
    statent_filtered_2[f'travel_time_{mode}'] = (
            (statent_filtered_2[f'path_length_{mode}'] / speed_factor)
    )
    
    statent_filtered_2[f'travel_time_access_egress_{mode}'] = (
            + ((access_cost * access_egress_detour_factor) / (walking_speed_kmh / 3.6))
            + ((statent_filtered_2[f'egress_{mode}'] * access_egress_detour_factor) / (walking_speed_kmh / 3.6))
    )

statent_filtered_2

In [ ]:
statent_filtered_2['from_cell'] = statent_id
statent_filtered_2

In [ ]:
# add euclidean distance

origin_cell = statent_filtered[statent_filtered['id'] == statent_id]
origin_geometry = min(origin_cell.geometry)
statent_filtered_2['euclidean_distance'] = statent_filtered_2.apply(
    lambda row: shp.distance(origin_geometry, row.geometry),
    axis=1
)
statent_filtered_2

In [ ]:
destinations_with_cost = pd.merge(
    statpop_reduced.reset_index(),
    statent_filtered_2.reset_index()[[
        'id', 'VOLLZEITAEQ_TOTAL',
        'euclidean_distance',
        'travel_time_foot',
        'travel_time_cycling',
        'travel_time_transit',
        'travel_time_private_cars',
        'travel_time_access_egress_foot',
        'travel_time_access_egress_cycling',
        'travel_time_access_egress_private_cars',
        'path_length_private_cars',
        'path_length_cycling',
        'path_length_foot',
        'closest_node_private_cars',
        'from_cell',
        'geometry'
    ]].rename(columns={'geometry': 'destination_geometry'}),
    how='left', left_on='statent_id', right_on='from_cell')
destinations_with_cost

In [ ]:
# ensure minimum travel time and euclidian distance
for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    destinations_with_cost[f'travel_time_{mode}'] = np.maximum(destinations_with_cost[f'travel_time_{mode}'], min_travel_time)

destinations_with_cost['euclidean_distance'] = np.maximum(destinations_with_cost['euclidean_distance'], min_euclidian_distance)

In [ ]:
#run the mode choice model to calculate each person's generalized cost to each statent destination
def calculate_behavioral_cost(
    euclidean_distance,
    travel_time_private_cars=np.inf, travel_time_access_egress_private_cars=np.inf, path_length_private_cars=np.inf,
    travel_time_transit=np.inf,
    travel_time_cycling=np.inf, travel_time_access_egress_cycling=np.inf,
    travel_time_foot=np.inf, travel_time_access_egress_foot=np.inf,
    age=None
):
    """
    Calculates individual generalized cost based on the personal properties and mode choice.
    Based on mode choice model implemented in eqasim.
    
    Returns a tuple of mode specific choice probabilities (dict) and the resulting behavioral travel time (float)
    """
    
    U = {}
    
    X_dist = euclidean_distance / 1000
    Mi_hhIncome = 12260
    X_hhIncome = 10000
    X_work = 0.5
    X_city_center = 0.2
    Mi_dist = 39
    Lambda_distTT = 0.1147
    Beta_cost = -0.088
    Lambda_dist_cost = -0.2209
    Lambda_hhIncome = -0.8169
    
    X_IVT_car = travel_time_private_cars / 60
    X_AET_car = travel_time_access_egress_private_cars / 60
    X_cost_car = 0.26 * path_length_private_cars / 1000
    Alpha_car = -0.8
    Beta_TT_car = -0.0192
    Beta_TT_walk = -0.0457
    Beta_work_car = -1.1606
    Beta_city_center_car = -0.4590
    
    U[MODE_PRIVATE_CARS] = (
        Alpha_car
        + Beta_TT_car * X_IVT_car * (X_dist / Mi_dist) ** Lambda_distTT
        + Beta_TT_walk * X_AET_car
        + Beta_cost * (X_dist / Mi_dist) ** Lambda_dist_cost + X_cost_car * (X_hhIncome / Mi_hhIncome) ** Lambda_hhIncome
        + Beta_work_car * X_work
        + Beta_city_center_car * X_city_center
    )
    
    X_railTT = travel_time_transit * 0.8 / 60  # surrogate
    X_busTT = travel_time_transit * 0.2 / 60 # surrogate
    X_AET_PT = 0 # = 0 because R5 includes it in the travel time
    X_waiting_time = 5 # surrogate
    X_number_of_connections = travel_time_transit / 60 / 20
    X_cost_PT = min(2.7, euclidean_distance / 1000 * 0.6)
    X_headway = 10 # surrogate
    Alpha_PT = 0
    Beta_railTT = -0.0072
    Beta_busTT = -0.0124
    Beta_AET_PT = -0.0142
    Beta_wait = -0.0124
    Beta_lineSwitch = -0.17
    Beta_headway = -0.0301
    Beta_OVGK = -0.8 # surrogate    
    
    U[MODE_TRANSIT] = (
        Alpha_PT
        + (Beta_railTT * X_railTT + Beta_busTT * X_busTT) * (X_dist / Mi_dist) ** Lambda_distTT
        + Beta_AET_PT * X_AET_PT
        + Beta_wait * X_waiting_time
        + Beta_lineSwitch * X_number_of_connections
        + Beta_cost * (X_dist / Mi_dist) ** Lambda_dist_cost * X_cost_PT * (X_hhIncome / Mi_hhIncome) ** Lambda_hhIncome
        + Beta_headway * X_headway
        + Beta_OVGK
    )
    
    X_bikeTT = travel_time_cycling / 60 + travel_time_access_egress_cycling / 60
    X_age = age
    Alpha_bike = -0.1258
    Beta_TT_bike = -0.1258
    Beta_age_60_plus_bike = -2.6588
    
    U[MODE_CYCLING]= (
        Alpha_bike
        + Beta_TT_bike * X_bikeTT + (X_dist / Mi_dist) ** Lambda_distTT
        + Beta_age_60_plus_bike * (X_age >= 60)
    )
    
    X_walkTT = travel_time_foot / 60 + travel_time_access_egress_foot / 60
    Alpha_walk = 0.5903
    Beta_TT_walk = -0.0457
    Theta_threshold_walkTT = 120
    
    U[MODE_FOOT] = (
        Alpha_walk
        + Beta_TT_walk * X_walkTT * (X_dist / Mi_dist) ** Lambda_distTT
        + (1 - 100 ** (X_walkTT / Theta_threshold_walkTT))
    )

    # replace nan with -inf for non-existent options (=infinite cost)
    U = {mode: -np.inf if np.isnan(U_mode) else U_mode for mode, U_mode in U.items()}

    # mode-specific choice probabilities
    denominator = sum([np.exp(U_mode) for U_mode in U.values()])
    if denominator != 0:
        
        P = {mode: np.exp(U_mode) / denominator for mode, U_mode in U.items()}
        #print(U)
        
        # mode-specific cost
        C = {mode: -U_mode for mode, U_mode in U.items()}
        
        # weighted cost
        C_weighted = sum([C[mode] * P[mode] if not np.isinf(C[mode]) else 0 for mode in P.keys()])
    
    else:
        
        P = {mode: None for mode, U_mode in U.items()}
        C = {mode: np.inf for mode, U_mode in U.items()}
        C_weighted = np.inf
        
    return P, C, C_weighted
        

destinations_with_cost[['choice_probabilities', 'cost', 'weighted_cost']] = destinations_with_cost.apply(
    lambda row: calculate_behavioral_cost(
        **row[[
            'euclidean_distance',
            'travel_time_private_cars', 'travel_time_access_egress_private_cars', 'path_length_private_cars',
            'travel_time_transit',
            'travel_time_cycling', 'travel_time_access_egress_cycling',
            'travel_time_foot', 'travel_time_access_egress_foot',
            'age'
        ]]
    ),
    axis=1,
    result_type='expand'
)

destinations_with_cost

In [ ]:
destinations_with_cost = pd.concat([
    destinations_with_cost,
    pd.json_normalize(destinations_with_cost['choice_probabilities']).add_prefix('p_'),
    pd.json_normalize(destinations_with_cost['cost']).add_prefix('c_'),
    ],
    axis=1
)

# calculate total accessibility contribution using cost function and destination utility
destinations_with_cost['accessibility_contribution'] = (
    (destinations_with_cost['weighted_cost'] ** -0.7) 
    * destinations_with_cost['VOLLZEITAEQ_TOTAL']
)

destinations_with_cost

In [ ]:
# calculate accessibility contribution for every mode separately
for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    destinations_with_cost[f'accessibility_contribution_{mode}'] = (
        (destinations_with_cost[f'c_{mode}'] ** -0.7)
        * destinations_with_cost['VOLLZEITAEQ_TOTAL']
    )
    
destinations_with_cost

In [ ]:
# for each person, sum the accessibility contributions across all destinations
accessibility = destinations_with_cost.groupby('record').agg({
    'age': 'first',
    'sex': 'first',
    'maritalstatus': 'first',
    'residencepermit': 'first',
    'residentpermit': 'first',
    'statent_id': 'first',
    'accessibility_contribution': 'sum',
    'accessibility_contribution_cycling': 'sum',
    'accessibility_contribution_foot': 'sum',
    'accessibility_contribution_private_cars': 'sum',
    'accessibility_contribution_transit': 'sum',
    'geometry': 'first'
})

accessibility.rename(columns={
    'accessibility_contribution': 'accessibility',
    'accessibility_contribution_cycling': 'accessibility_cycling',
    'accessibility_contribution_foot': 'accessibility_foot',
    'accessibility_contribution_private_cars': 'accessibility_private_cars',
    'accessibility_contribution_transit': 'accessibility_transit',
}, inplace=True)

accessibility.reset_index(inplace=True)
accessibility = gpd.GeoDataFrame(accessibility, crs=2056)
accessibility

In [ ]:
if 0:
    snman.io.export_gdf(
        accessibility,
        os.path.join(outputs_path, f'accessibility_{STATE}.gpkg')
    )

In [ ]:
accessibility_contributions = copy.deepcopy(destinations_with_cost)
accessibility_contributions['geometry'] = accessibility_contributions.apply(
    lambda row: shp.LineString([row['geometry'], row['destination_geometry']]),
    axis=1
)
accessibility_contributions['geometry'] = accessibility_contributions['destination_geometry']
accessibility_contributions.drop(columns=['destination_geometry'], inplace=True)

snman.io.export_gdf(
    accessibility_contributions,
    os.path.join(outputs_path, f'accessibility_contributions_{STATE}.gpkg')
)

accessibility_contributions

In [ ]:
STATE

In [ ]:
# testbed for point to point routing

# node IDs
#origin, destination = (1756, 51785) # cross-city before
origin, destination = (50608, 1776) # cross-city after
#origin, destination = (50608, 35180)
#weight = 'matsim_tt_cars'
weight = 'cost_cycling'

L = L_modes[MODE_CYCLING]
route = nx.shortest_path(L, origin, destination, weight=weight)

uvk_and_order = {}
for u, v in zip(route[:-1], route[1:]):
    k, data = min(L.get_edge_data(u, v).items(), key=lambda e: e[1][weight] + (e[1]['lane'].lanetype == LANETYPE_MOTORIZED))
    uvk_and_order[(u,v,k)] = len(uvk_and_order)
    
M = L.edge_subgraph(uvk_and_order.keys())
nx.set_edge_attributes(M, uvk_and_order, 'order_in_path')
M = snman.lane_graph.LaneGraph(M)

edges = oxc.graph_to_gdfs(M, nodes=False)
print(edges.groupby('lanetype').agg({weight: 'sum', 'length': 'sum'}))
print(edges.agg({weight: 'sum', 'length': 'sum'}))

snman.io.export_lane_geometries(
    M,
    os.path.join(outputs_path, f'path_edges.gpkg'),
    os.path.join(outputs_path, f'path_nodes.gpkg'),
    scaling=30
)

In [ ]:
STATE